<a href="https://colab.research.google.com/github/ManantenaKiady/Pytorch-fundamentals/blob/master/Notebooks/Pytorch_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Setting up Pytorch 1.13.0, the latest stable release along with other needed libraries and packages. 

### 1- Installation

To install Pytorch

* Go to https://pytorch.org/get-started/locally/ and download the latest Pytorch release. Follows the instructions

In [1]:
!pip3 install torch torchvision torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Check the installed version 
import torch
torch.__version__

'1.13.0+cu116'

### 2- Configuration

In [3]:
# Check if GPU are availables and set the device to use it
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [20]:
# Create a folder named models to store trained models
!mkdir models

mkdir: cannot create directory ‘models’: File exists


# The Basics with Pytorch

### 1- Working with data

Load data using Pytorch

In [4]:
import torch 
from torch.utils.data import DataLoader
# Here we demonstrate an example with a common dataset from TorchVision ( Computer Vision )
from torchvision import datasets 
from torchvision.transforms import ToTensor

In [5]:
# To retrieve the dataset
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [6]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

We can pass the Dataset as an argument to DataLoader in order to:

- Make the Dataset iterable
- To supports automatic batching and sampling
- To make it shuffleable 
- To support multiprocess data loading

In [7]:
# Define a batch size
# HINT: Batch Size: The number of features and labels (examples) feed in one iteration.

BATCH_SIZE = 32

# Shape
C = 0
H = 0
W = 0

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)

for X, y in train_dataloader:
  # N: Batch Size, C: Number of Channels of the image ( Here 1 Gray), H: Height, W: Width
  print(f"Shape of X [N,C,H,W]: {X.shape}")
  # Retireve shape from data
  [_,C,H,W] = X.shape
  print(f"Shape of Y: {y.shape}, of type {y.dtype}")
  break

Shape of X [N,C,H,W]: torch.Size([32, 1, 28, 28])
Shape of Y: torch.Size([32]), of type torch.int64


### 2- Your first Model

- Create a class that inherits from torch.nn.Module.
- Network Layers should be defined under the ```__init__``` function.
- Add a ```forward``` method and specify how the data will pass throught the network.

In [8]:
from torch import nn 

In [9]:

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(H*W, 512), # Linear Layer
        nn.ReLU(), # Apply a Relu Activation Function
        nn.Linear(512,512),
        nn.ReLU(),
        # Here the output will be 0 to 9, thus of size of 10
        nn.Linear(512,10)
    )

  def forward(self, X):
    x = self.flatten(X)
    logits = self.linear_relu_stack(x)
    return logits


In [10]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### 3- Optimization

Optimizing the Model Parameters ( The Learning Algorithm )

**The Loss function**: The function to optimize (minimize)

In [11]:
loss_fn = nn.CrossEntropyLoss()
# Define the optimization algorithm
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [12]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X,y) in enumerate(dataloader):
    # Set the data to load into the device
    X,y = X.to(device), y.to(device)

    # Compute the prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    # Set the gradients to all Tensor to zero
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

Check the performance of the model against the test datasets ( Validation dataset)


In [13]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)

  model.eval()
  test_loss, correct = 0, 0
  # Disable gradient calculation
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  
  test_loss /= num_batches 
  correct /= size 

  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


**Training epochs**
The number of iterations to train the model, for each iteration, whole dataset is passed through the model.

In [14]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n ------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1
 ------------------------
loss: 2.312187 [    0/60000]
loss: 2.284373 [ 3200/60000]
loss: 2.282747 [ 6400/60000]
loss: 2.270216 [ 9600/60000]
loss: 2.246768 [12800/60000]
loss: 2.233394 [16000/60000]
loss: 2.221734 [19200/60000]
loss: 2.198668 [22400/60000]
loss: 2.209325 [25600/60000]
loss: 2.150463 [28800/60000]
loss: 2.112041 [32000/60000]
loss: 2.105152 [35200/60000]
loss: 2.175808 [38400/60000]
loss: 2.077159 [41600/60000]
loss: 2.038897 [44800/60000]
loss: 2.029630 [48000/60000]
loss: 2.072700 [51200/60000]
loss: 1.979783 [54400/60000]
loss: 1.933673 [57600/60000]
Test Error: 
 Accuracy: 54.6%, Avg loss: 1.919449 

Epoch 2
 ------------------------
loss: 1.892679 [    0/60000]
loss: 1.911510 [ 3200/60000]
loss: 1.863752 [ 6400/60000]
loss: 1.820704 [ 9600/60000]
loss: 1.753813 [12800/60000]
loss: 1.811130 [16000/60000]
loss: 1.634404 [19200/60000]
loss: 1.618715 [22400/60000]
loss: 1.628096 [25600/60000]
loss: 1.596299 [28800/60000]
loss: 1.441578 [32000/60000]
loss: 1.38

### 4- Saving Models

In [26]:

torch.save(model.state_dict(), "models/model_states.pth")
print("Saved Model State to model_states.pth")

Saved Model State to model_states.pth


**Questions:** Can I save the model structure along the dict states ? 

**Answer**: Yeah, you can

In [40]:
torch.save(model, "models/model.pt") # By convention we use .pt or .pth 
print("Saved Entire Model to model.pt")

Saved Entire Model to model.pt


### 5- Loading Models
- Recreating the model structure or architecture
- Loading the weights or the state dictionary into it

In [39]:
model = NeuralNetwork()
model.load_state_dict(torch.load("models/model_states.pth"))

<All keys matched successfully>

**Loading Entire model**

In [42]:
model=torch.load("models/model.pt")

### 5- Predictions

In [43]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

# Set the model (nn Module) in evaluation mode
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f"Predicted: {predicted}, Actual: {actual}")

Predicted: Ankle boot, Actual: Ankle boot
